<a href="https://colab.research.google.com/github/maheshkulkarni123OS/Freelance-Work/blob/master/Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NVCC Installation is done via another notebook

In [ ]:
!/usr/local/cuda/bin/nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
from google.colab import drive 
drive.mount('/content/gdrive')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-t16xbxwv
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-t16xbxwv
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=c6e60670315720ff3e8240c11f0d47f75fcd7dd846f0ae7a5ef31c47a8cb5e8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-6j8leey5/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.

Sample code to check whether CUDA is running properly or not

In [ ]:
%%cu
#include "stdio.h"
__global__ void add(int a, int b, int *c)
{
  printf("hello");
 *c = a + b;
}
int main()
{
int a,b,c;
int *dev_c;
a=3;
b=4;
cudaMalloc((void**)&dev_c, sizeof(int));
add<<<1,1>>>(a,b,dev_c);
cudaMemcpy(&c, dev_c, sizeof(int), cudaMemcpyDeviceToHost);
printf("%d + %d is %d\n", a, b, c);
cudaFree(dev_c);
return 0;
}


hello3 + 4 is 7



In [ ]:
%%cu

//Main Code
#include<stdio.h>
#define N 4048
char buffer[20];
char Date[10000][25];
float open[10000];
float low[10000];
float high[10000];
float close[10000];
float adj_close[10000];
float volm[10000];
int count = 0,horizon=0; 
int num_of_periods_for_mov_avg = 10;
float ema[10000]={0};
float wma[10000]={0};
float mfr[1000]={0.0},mfi[1000]={0.0},rsi[1000]={0.0},rs[1000] = {0.0};
float tr[1000] = {0.0},atr[1000] = {0.0},per_r[1000] = {0.0};
int weight_array[40] = {0};
float typical_price[1000]={0.0},mon_flow[1000]={0.0};
float expa=2;
//float *dev_b;
float smaa[N];

__global__ void sum(int* input)
{
	const int tid = threadIdx.x;

	auto step_size = 1;
	int number_of_threads = blockDim.x;

	while (number_of_threads > 0)
	{
		if (tid < number_of_threads) // still alive?
		{
			const auto fst = tid * step_size * 2;
			const auto snd = fst + step_size;
			input[fst] += input[snd];
		}

		step_size <<= 1; 
		number_of_threads >>= 1;
	}
 printf("SUMATION =  %d",input[0]);
}

void MFI()
  {
   float *dev_a;
   float *dev_b;
   int *dev_c,*dev_d; 
   int size = N * sizeof( float ); 
   int size1 = N * sizeof( int ); 
   cudaMalloc( (void**)&dev_a, size );
   cudaMalloc( (void**)&dev_b, size );
   cudaMalloc( (void**)&dev_c, size );
   cudaMalloc( (void**)&dev_d, sizeof(int) );

   cudaMemcpy( dev_a, typical_price, size, cudaMemcpyHostToDevice );
   cudaMemcpy( dev_b, close, size, cudaMemcpyHostToDevice );
   cudaMemcpy( dev_c, high, size, cudaMemcpyHostToDevice );
   cudaMemcpy( dev_d, low, size, cudaMemcpyHostToDevice );
   calc_wma<<< 1, 192 >>>( dev_b, dev_c,dev_d,dev_a );
  }

void MFI()
 {
   int i,j;
   float typical_price[1000]={0.0},mon_flow[1000]={0.0};
   float pos_mon_flow=0.0,neg_mon_flow=0.0;
   for(i=0;i<count;i++)
    {
      typical_price[i] = (high[i]+low[i]+close[i])/3;
      mon_flow[i] = typical_price[i] * (volm[i] /10000);
    }
   for(j=0;j<count-14;j++)
    {
      pos_mon_flow=0.0;
      neg_mon_flow=0.0;
     for(i=j+1;i<j+15;i++)
      {
       {
         if(typical_price[i-1]>typical_price[i])
           neg_mon_flow += mon_flow[i];
         else
           pos_mon_flow += mon_flow[i];
        }
      }
       mfr[i-1] = pos_mon_flow/neg_mon_flow;
       mfi[i-1] = (100*mfr[i-1]) / (1+mfr[i-1]);
    }
 }

__global__ void calc_wma( float *close, int *weight_array,int *weight_array_sum, float *wma ) 
 {
   float avg=0; 
   int i,k;
  //call sum
  /*const auto count = 8;
	const int size = count * sizeof(int);
	int h[] = {13, 27, 15, 14, 33, 2, 24, 6};

	int* d;
	
	cudaMalloc(&d, size);
	cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);

	sum <<<1, count / 2 >>>(d);

	int result;
	cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);

	cout << "Sum is " << result << endl;*/
   for(i =threadIdx.x, k=0;i<threadIdx.x+10;i++,k++)
      avg = avg+close[i] * weight_array[k];
    wma[threadIdx.x +10 -1] = avg / *weight_array_sum;
 }

int weight_array_initialise()
  {
   int sum;
   for(int i=0;i<num_of_periods_for_mov_avg;i++)
     {
       weight_array[i] = i;
     }
   for(int i=0;i<num_of_periods_for_mov_avg;i++)
     sum = sum + weight_array[i];
   return sum;
  }

void wmaa()
 {
   int weight_array_sum;
   weight_array_sum = weight_array_initialise();
   float *dev_a;
   float *dev_b;
   int *dev_c,*dev_d; 
   int size = N * sizeof( float ); 
   int size1 = N * sizeof( int ); 
   cudaMalloc( (void**)&dev_a, size );
   cudaMalloc( (void**)&dev_b, size );
   cudaMalloc( (void**)&dev_c, size );
   cudaMalloc( (void**)&dev_d, sizeof(int) );

   cudaMemcpy( dev_a, wma, size, cudaMemcpyHostToDevice );
   cudaMemcpy( dev_b, close, size, cudaMemcpyHostToDevice );
   cudaMemcpy( dev_c, weight_array, size1, cudaMemcpyHostToDevice );
   cudaMemcpy( dev_d, &weight_array_sum, sizeof(int), cudaMemcpyHostToDevice );
   calc_wma<<< 1, 192 >>>( dev_b, dev_c,dev_d,dev_a );
   cudaMemcpy( wma, dev_a, size, cudaMemcpyDeviceToHost );
   cudaFree( dev_a );   
   cudaFree( dev_b );
   cudaFree( dev_c );
   cudaFree ( dev_d );
 }

__global__ void calc_sma( float *close, float *smaa ) 
 {
   float avg=0; 
   for(int i =threadIdx.x;i<threadIdx.x+10;i++)
      avg+=close[i];
    smaa[threadIdx.x +10 -1] = avg/10;
 }

void sma()
  {
      float *dev_c; 
      float *dev_b;
      int size = N * sizeof( float ); 
      cudaMalloc( (void**)&dev_b, size );
      cudaMalloc( (void**)&dev_c, size );

      cudaMemcpy( dev_b, close, size, cudaMemcpyHostToDevice );

      calc_sma<<< 1, 192 >>>( dev_b, dev_c );
      cudaMemcpy( smaa, dev_c, size, cudaMemcpyDeviceToHost );
      
      cudaFree( dev_b );
      cudaFree( dev_c );
  }

void put_value_in_array(float value,int select_array)
   {
     switch(select_array)
       {
         case 1:
           open[count] = value;
           break;
         case 2:
           high[count] = value;
           break;
         case 3:
           low[count] = value;
           break;
         case 4:
           close[count] = value;
           break;
         case 5:
           volm[count] = value;
           break;
       }
   }

float converting_char_array_into_float()
 {
   float num=0,num1,num2;
   int i=0,mul=1;
   while(buffer[i]!='.')
    {
     num2 = (float)buffer[i]-48;
     num = num*10+num2;
     i++;
     if(buffer[i]==13)
      {
       return num;
      }
    }
   i++;
   while(buffer[i]!='\0')
    {
     num2=(float)buffer[i]-48;
     mul=mul*10;
     num1=num2/mul;
     num=num+num1;
     i++;
    }
   return num;
 }

void reinitialise_buffer_to_zero()
  {
    for(int i=0;i<20;i++)
      {
	       buffer[i] = '\0';
      }
  }

void put_value_in_buffer(char code)
  {
    buffer[horizon] = code;
    horizon++;
  }

void put_in_date_array()
   {
     for(int i=0;i<20;i++)
      {
	      Date[count][i] = buffer[i]; 
      }
   }

int main( void ) 
{
  FILE *file;
    //float *dev_b;
    int size = N * sizeof( float );
    //cudaMalloc( (void**)&dev_b, size );
    //cudaMemcpy( dev_b, close, size, cudaMemcpyHostToDevice );
    float value;
    int select_array=0;
    char code;
    file = fopen("gdrive/My Drive/export_csv1.csv","r");
    code = (char)fgetc(file);
    while(code!='\n')
      {
       printf("%c",code);
       code = (char)fgetc(file);
      }
   do
     {
       code = (char)fgetc(file);
       if(code==',')
        {
          if(select_array==0)
             {
		            select_array++;
                put_in_date_array();
                reinitialise_buffer_to_zero();
                horizon = 0;
             }
          else
            {
              value = converting_char_array_into_float();
              put_value_in_array(value,select_array);
              select_array++;
              reinitialise_buffer_to_zero();
              horizon =0;
	          }
        }
       else if (code=='\n')
        {
          value = converting_char_array_into_float();
          put_value_in_array(value,select_array);
          select_array=0;
          count++;
          horizon = 0;
        }
       else
        {
       	  put_value_in_buffer(code);
        }
     }while(code != EOF);
  sma();
  wmaa();
  for(int i=0;i<count;i++)
    {
      for(int j=0;j<19;j++)
         {
           printf("%c",Date[i][j]);
         }
          printf("\t%f",smaa[i]);
          printf("\t%f\n",wma[i]);
    }
  //cudaFree( dev_b );
  return 0;
}

/tmp/tmp9oc0p68z/1603812c-0429-4068-aa4e-9c0a979a403f.cu(63): error: identifier "calc_wma" is undefined

/tmp/tmp9oc0p68z/1603812c-0429-4068-aa4e-9c0a979a403f.cu(53): warning: variable "size1" was declared but never referenced

/tmp/tmp9oc0p68z/1603812c-0429-4068-aa4e-9c0a979a403f.cu(66): error: function "MFI" has already been defined

/tmp/tmp9oc0p68z/1603812c-0429-4068-aa4e-9c0a979a403f.cu(256): warning: variable "size" was declared but never referenced

2 errors detected in the compilation of "/tmp/tmpxft_00001119_00000000-8_1603812c-0429-4068-aa4e-9c0a979a403f.cpp1.ii".



In [ ]:
#retry
%%cu
#include<stdio.h>
char buffer[20];
char Date[10000][25];
float open[10000];
float low[10000];
float high[10000];
float close[10000];
float adj_close[10000];
float volm[10000];
int count = 0,horizon=0; 
int num_of_periods_for_mov_avg = 10;
float sma[10000]={0};	
float ema[10000]={0};
float wma[10000]={0};
float mfr[1000]={0.0},mfi[1000]={0.0},rsi[1000]={0.0},rs[1000] = {0.0};
float tr[1000] = {0.0},atr[1000] = {0.0},per_r[1000] = {0.0};
int weight_array[40] = {0};
float expa=2;
int main()
  {
    FILE *file;
    float value;
    int select_array=0;
    char code;
    file = fopen("gdrive/My Drive/export_csv1.csv","r");
    code = (char)fgetc(file);
    while(code!='\n')
      {
       printf("%c",code);
       code = (char)fgetc(file);
      }
   do
     {
       code = (char)fgetc(file);
       if(code==',')
        {
          if(select_array==0)
             {
		            select_array++;
                put_in_date_array();
                reinitialise_buffer_to_zero();
                horizon = 0;
             }
          else
            {
              value = converting_char_array_into_float();
              put_value_in_array(value,select_array);
              select_array++;
              reinitialise_buffer_to_zero();
              horizon =0;
	          }
        }
       else if (code=='\n')
        {
          value = converting_char_array_into_float();
          put_value_in_array(value,select_array);
          select_array=0;
          count++;
          horizon = 0;
        }
       else
        {
       	  put_value_in_buffer(code);
        }
     }while(code != EOF);
   for(int i=0;i<count;i++)
     {
       for(int j=0;j<19;j++)
         {
           printf("%c",Date[i][j]);
         }
        printf("\t");
        printf("%f\t",open[i]);
        printf("%f\t",high[i]);
        printf("%f\t",low[i]);
        printf("%f\t",close[i]);
        printf("%f\n",volm[i]);
     }
   printf("\n\n");
   smaa();
   emaa();
   wmaa();
   //ADX();
   MFI();
   RSI();
   ATR();
   willi_indi();
   printf("\n\n");
   print();
   fclose(file);
   file = fopen("gdrive/My Drive/export_csv2.csv","w+");
   fprintf(file,"Timestamp,open,high,low,close,volm,sma,ema,wma,mfr,mfi,rsi,rs,tr,atr,per_r");
   for(int i=0;i<count;i++)
    {
      fprintf(file,"\n%s,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f,%f",Date[i],open[i],high[i],low[i],close[i],volm[i],sma[i],ema[i],wma[i],mfr[i],mfi[i],rsi[i],rs[i],tr[i],atr[i],per_r[i]);
    }
   return 0;
  }

In [ ]:
#global Variable
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <iostream>
#include <numeric>
using namespace std;

__device__ int count;  // Declare global variable here
__device__ void  calldevicefunction ( ) {

  count = count +1; // use the global variable directly here.

}
__global__ void core_kernel(){

  calldevicefunction(); // global function calls the device function

}


int  main() {

do {

  core_kernel<<<5, 32>>>();  //Call the CUDA kernel

  //// What is the value in count variable here???

  } while (count != 16);
return 0;
}



KeyboardInterrupt: ignored

In [ ]:
%%cu
#include <cstdio>

#define nn (10)

__constant__ float a[nn];

__global__ void kernel(float *out)
{
    if (threadIdx.x < nn)
        out[threadIdx.x] = a[threadIdx.x];

}

int main()
{
    const size_t sz = size_t(nn) * sizeof(float);
    const float avals[nn]={ 1., 2., 3., 4., 5., 6., 7., 8., 9., 10. };
    float ah[nn];

    cudaMemcpyToSymbol("a", &avals[0], sz, size_t(0),cudaMemcpyHostToDevice);

    float *ad;
    cudaMalloc((void **)&ad, sz);

    kernel<<<dim3(1),dim3(16)>>>(ad);

    cudaMemcpy(&ah[0],ad,sz,cudaMemcpyDeviceToHost);

    for(int i=0; i<nn; i++) {
        printf("%d %f\n", i, ah[i]);
    }
}

0 0.000000
1 0.000000
2 0.000000
3 0.000000
4 0.000000
5 0.000000
6 0.000000
7 0.000000
8 0.000000
9 0.000000



In [ ]:
%%cu

#include <stdio.h>
#define N 4048

char buffer[20];
char Date[10000][25];
float open[10000];
float low[10000];
float high[10000];
float close[10000];
float adj_close[10000];
float volm[10000];
int count = 0,horizon=0; 
int num_of_periods_for_mov_avg = 10;
float ema[10000]={0};
float wma[10000]={0};
float mfr[1000]={0.0},mfi[1000]={0.0},rsi[1000]={0.0},rs[1000] = {0.0};
float tr[1000] = {0.0},atr[1000] = {0.0},per_r[1000] = {0.0};
int weight_array[40] = {0};
float expa=2;
float smaa[N];
__global__ void add( float *close, float *smaa ) {
   float avg=0; 
   for(int i =threadIdx.x;i<threadIdx.x+10;i++)
      avg+=close[i];
    smaa[threadIdx.x +10 -1] = avg/10;
}
void sma()
  {
      float *dev_b, *dev_c; 
      int size = N * sizeof( float ); 
      cudaMalloc( (void**)&dev_b, size );
      cudaMalloc( (void**)&dev_c, size );

      cudaMemcpy( dev_b, close, size, cudaMemcpyHostToDevice );

      add<<< 1, 192 >>>( dev_b, dev_c );
      cudaMemcpy( smaa, dev_c, size, cudaMemcpyDeviceToHost );
      
      cudaFree( dev_b );
      cudaFree( dev_c );
  }
void put_value_in_array(float value,int select_array)
   {
     switch(select_array)
       {
         case 1:
           open[count] = value;
           break;
         case 2:
           high[count] = value;
           break;
         case 3:
           low[count] = value;
           break;
         case 4:
           close[count] = value;
           break;
         case 5:
           volm[count] = value;
           break;
       }
   }

float converting_char_array_into_float()
 {
   float num=0,num1,num2;
   int i=0,mul=1;
   while(buffer[i]!='.')
    {
     num2 = (float)buffer[i]-48;
     num = num*10+num2;
     i++;
     if(buffer[i]==13)
      {
       return num;
      }
    }
   i++;
   while(buffer[i]!='\0')
    {
     num2=(float)buffer[i]-48;
     mul=mul*10;
     num1=num2/mul;
     num=num+num1;
     i++;
    }
   return num;
 }

void reinitialise_buffer_to_zero()
  {
    for(int i=0;i<20;i++)
      {
         buffer[i] = '\0';
      }
  }

void put_value_in_buffer(char code)
  {
    buffer[horizon] = code;
    horizon++;
  }

void put_in_date_array()
   {
     for(int i=0;i<20;i++)
      {
        Date[count][i] = buffer[i]; 
      }
   }

int main( void ) 
{
		printf("Print0");
  FILE *file;
    float value;
    int select_array=0;
    char code;
    file = fopen("gdrive/My Drive/export_csv1.csv","r");
    code = (char)fgetc(file);
    while(code!='\n')
      {
       printf("%c",code);
       code = (char)fgetc(file);
      }
   do
     {
       code = (char)fgetc(file);
       if(code==',')
        {
          if(select_array==0)
             {
                select_array++;
                put_in_date_array();
                reinitialise_buffer_to_zero();
                horizon = 0;
             }
          else
            {
              value = converting_char_array_into_float();
              put_value_in_array(value,select_array);
              select_array++;
              reinitialise_buffer_to_zero();
              horizon =0;
            }
        }
       else if (code=='\n')
        {
          value = converting_char_array_into_float();
          put_value_in_array(value,select_array);
          select_array=0;
          count++;
          horizon = 0;
        }
       else
        {
          put_value_in_buffer(code);
        }
     }while(code != EOF);
  printf("Print0");
	sma();
	printf("Print1");
  for(int i=0;i<count;i++)
    {
      for(int j=0;j<19;j++)
         {
           printf("%c",Date[i][j]);
         }
          printf("\t%f\n",smaa[i]);
    }
  return 0;
}


In [ ]:
%%cu
#include<stdio.h>
#define N 4048
char buffer[20];
char Date[10000][25];
float open[10000];
float low[10000];
float high[10000];
float close[10000];
float adj_close[10000];
float volm[10000];
int count = 0,horizon=0; 
int num_of_periods_for_mov_avg = 10;
float ema[10000]={0};
float wma[10000]={0};
float mfr[1000]={0.0},mfi[1000]={0.0},rsi[1000]={0.0},rs[1000] = {0.0};
float tr[1000] = {0.0},atr[1000] = {0.0},per_r[1000] = {0.0};
int weight_array[40] = {0};
float expa=2;
float smaa[N];
__global__ void add( float *close, float *smaa ) {
   float avg=0; 
   for(int i =threadIdx.x;i<threadIdx.x+10;i++)
      avg+=close[i];
    smaa[threadIdx.x +10 -1] = avg/10;
}
void sma()
  {
      float *dev_b, *dev_c; 
      int size = N * sizeof( float ); 
      cudaMalloc( (void**)&dev_b, size );
      cudaMalloc( (void**)&dev_c, size );

      cudaMemcpy( dev_b, close, size, cudaMemcpyHostToDevice );

      add<<< 1, 192 >>>( dev_b, dev_c );
      cudaMemcpy( smaa, dev_c, size, cudaMemcpyDeviceToHost );
      
      cudaFree( dev_b );
      cudaFree( dev_c );
  }
void put_value_in_array(float value,int select_array)
   {
     switch(select_array)
       {
         case 1:
           open[count] = value;
           break;
         case 2:
           high[count] = value;
           break;
         case 3:
           low[count] = value;
           break;
         case 4:
           close[count] = value;
           break;
         case 5:
           volm[count] = value;
           break;
       }
   }

float converting_char_array_into_float()
 {
   float num=0,num1,num2;
   int i=0,mul=1;
   while(buffer[i]!='.')
    {
     num2 = (float)buffer[i]-48;
     num = num*10+num2;
     i++;
     if(buffer[i]==13)
      {
       return num;
      }
    }
   i++;
   while(buffer[i]!='\0')
    {
     num2=(float)buffer[i]-48;
     mul=mul*10;
     num1=num2/mul;
     num=num+num1;
     i++;
    }
   return num;
 }

void reinitialise_buffer_to_zero()
  {
    for(int i=0;i<20;i++)
      {
         buffer[i] = '\0';
      }
  }

void put_value_in_buffer(char code)
  {
    buffer[horizon] = code;
    horizon++;
  }

void put_in_date_array()
   {
     for(int i=0;i<20;i++)
      {
        Date[count][i] = buffer[i]; 
      }
   }

int main( void ) 
{
    printf("1");
  FILE *file;
    float value;
    int select_array=0;
    char code;
    file = fopen("gdrive/My Drive/export_csv1.csv","r");
    code = (char)fgetc(file);
    while(code!='\n')
      {
       printf("%c",code);
       code = (char)fgetc(file);
      }

   do
     {
       code = (char)fgetc(file);
       if(code==',')
        {
          if(select_array==0)
             {
                select_array++;
                put_in_date_array();
                reinitialise_buffer_to_zero();
                horizon = 0;
             }
          else
            {
              value = converting_char_array_into_float();
              put_value_in_array(value,select_array);
              select_array++;
              reinitialise_buffer_to_zero();
              horizon =0;
            }
        }
       else if (code=='\n')
        {
          value = converting_char_array_into_float();
          put_value_in_array(value,select_array);
          select_array=0;
          count++;
          horizon = 0;
        }
       else
        {
          put_value_in_buffer(code);
        }
     }while(code != EOF);
  sma();
/*  for(int i=0;i<count;i++)
    {
      for(int j=0;j<19;j++)
         {
           printf("%c",Date[i][j]);
         }
          printf("\t%f\n",smaa[i]);
    }*/
  return 0;
}

In [ ]:
%%cu
#include<stdio.h>
#define N 4048
char buffer[20];
char Date[10000][25];
float open[10000];
float low[10000];
float high[10000];
float close[10000];
float adj_close[10000];
float volm[10000];
int count = 0,horizon=0; 
int num_of_periods_for_mov_avg = 10;
float ema[10000]={0};
float wma[10000]={0};
float mfr[1000]={0.0},mfi[1000]={0.0},rsi[1000]={0.0},rs[1000] = {0.0};
float tr[1000] = {0.0},atr[1000] = {0.0},per_r[1000] = {0.0};
int weight_array[40] = {0};
float expa=2;
float smaa[N];
__global__ void add( float *close, float *smaa ) {
   float avg=0; 
   for(int i =threadIdx.x;i<threadIdx.x+10;i++)
      avg+=close[i];
    smaa[threadIdx.x +10 -1] = avg/10;
}
void sma()
  {
      float *dev_b, *dev_c; 
      int size = N * sizeof( float ); 
      cudaMalloc( (void**)&dev_b, size );
      cudaMalloc( (void**)&dev_c, size );

      cudaMemcpy( dev_b, close, size, cudaMemcpyHostToDevice );

      add<<< 1, 192 >>>( dev_b, dev_c );
      cudaMemcpy( smaa, dev_c, size, cudaMemcpyDeviceToHost );
      
      cudaFree( dev_b );
      cudaFree( dev_c );
  }
void put_value_in_array(float value,int select_array)
   {
     switch(select_array)
       {
         case 1:
           open[count] = value;
           break;
         case 2:
           high[count] = value;
           break;
         case 3:
           low[count] = value;
           break;
         case 4:
           close[count] = value;
           break;
         case 5:
           volm[count] = value;
           break;
       }
   }

float converting_char_array_into_float()
 {
   float num=0,num1,num2;
   int i=0,mul=1;
   while(buffer[i]!='.')
    {
     num2 = (float)buffer[i]-48;
     num = num*10+num2;
     i++;
     if(buffer[i]==13)
      {
       return num;
      }
    }
   i++;
   while(buffer[i]!='\0')
    {
     num2=(float)buffer[i]-48;
     mul=mul*10;
     num1=num2/mul;
     num=num+num1;
     i++;
    }
   return num;
 }

void reinitialise_buffer_to_zero()
  {
    for(int i=0;i<20;i++)
      {
         buffer[i] = '\0';
      }
  }

void put_value_in_buffer(char code)
  {
    buffer[horizon] = code;
    horizon++;
  }

void put_in_date_array()
   {
     for(int i=0;i<20;i++)
      {
        Date[count][i] = buffer[i]; 
      }
   }

int main( void ) 
{
  FILE *file;
    float value;
    int select_array=0;
    char code;
    file = fopen("gdrive/My Drive/export_csv1.csv","r");
    code = (char)fgetc(file);
    while(code!='\n')
      {
       printf("%c",code);
       code = (char)fgetc(file);
      }
   do
     {
       code = (char)fgetc(file);
       if(code==',')
        {
          if(select_array==0)
             {
                select_array++;
                put_in_date_array();
                reinitialise_buffer_to_zero();
                horizon = 0;
             }
          else
            {
              value = converting_char_array_into_float();
              put_value_in_array(value,select_array);
              select_array++;
              reinitialise_buffer_to_zero();
              horizon =0;
            }
        }
       else if (code=='\n')
        {
          value = converting_char_array_into_float();
          put_value_in_array(value,select_array);
          select_array=0;
          count++;
          horizon = 0;
        }
       else
        {
          put_value_in_buffer(code);
        }
     }while(code != EOF);
  sma();
  for(int i=0;i<count;i++)
    {
      for(int j=0;j<19;j++)
         {
           printf("%c",Date[i][j]);
         }
          printf("\t%f\n",smaa[i]);
    }
  return 0;
}

In [ ]:
%%cu
#include<stdio.h>
#define N 4048
char buffer[20];
char Date[10000][25];
float open[10000];
float low[10000];
float high[10000];
float close[10000];
float adj_close[10000];
float volm[10000];
int count = 0,horizon=0; 
int num_of_periods_for_mov_avg = 10;
float ema[10000]={0};
float wma[10000]={0};
float mfr[1000]={0.0},mfi[1000]={0.0},rsi[1000]={0.0},rs[1000] = {0.0};
float tr[1000] = {0.0},atr[1000] = {0.0},per_r[1000] = {0.0};
int weight_array[40] = {0};
float expa=2;
float smaa[N];
__global__ void add( float *close, float *smaa ) {
   float avg=0; 
   for(int i =threadIdx.x;i<threadIdx.x+10;i++)
      avg+=close[i];
    smaa[threadIdx.x +10 -1] = avg/10;
}
void sma()
  {
      float *dev_b, *dev_c; 
      int size = N * sizeof( float ); 
      cudaMalloc( (void**)&dev_b, size );
      cudaMalloc( (void**)&dev_c, size );

      cudaMemcpy( dev_b, close, size, cudaMemcpyHostToDevice );

      add<<< 1, 192 >>>( dev_b, dev_c );
      cudaMemcpy( smaa, dev_c, size, cudaMemcpyDeviceToHost );
      
      cudaFree( dev_b );
      cudaFree( dev_c );
  }
void put_value_in_array(float value,int select_array)
   {
     switch(select_array)
       {
         case 1:
           open[count] = value;
           break;
         case 2:
           high[count] = value;
           break;
         case 3:
           low[count] = value;
           break;
         case 4:
           close[count] = value;
           break;
         case 5:
           volm[count] = value;
           break;
       }
   }

float converting_char_array_into_float()
 {
   float num=0,num1,num2;
   int i=0,mul=1;
   while(buffer[i]!='.')
    {
     num2 = (float)buffer[i]-48;
     num = num*10+num2;
     i++;
     if(buffer[i]==13)
      {
       return num;
      }
    }
   i++;
   while(buffer[i]!='\0')
    {
     num2=(float)buffer[i]-48;
     mul=mul*10;
     num1=num2/mul;
     num=num+num1;
     i++;
    }
   return num;
 }

void reinitialise_buffer_to_zero()
  {
    for(int i=0;i<20;i++)
      {
         buffer[i] = '\0';
      }
  }

void put_value_in_buffer(char code)
  {
    buffer[horizon] = code;
    horizon++;
  }

void put_in_date_array()
   {
     for(int i=0;i<20;i++)
      {
        Date[count][i] = buffer[i]; 
      }
   }

int main( void ) 
{
    printf("Hello");
  FILE *file;
    float value;
    int select_array=0;
    char code;
    file = fopen("gdrive/My Drive/export_csv1.csv","r");
    code = (char)fgetc(file);
    while(code!='\n')
      {
       printf("%c",code);
       code = (char)fgetc(file);
      }
   do
     {
       code = (char)fgetc(file);
       if(code==',')
        {
          if(select_array==0)
             {
                select_array++;
                put_in_date_array();
                reinitialise_buffer_to_zero();
                horizon = 0;
             }
          else
            {
              value = converting_char_array_into_float();
              put_value_in_array(value,select_array);
              select_array++;
              reinitialise_buffer_to_zero();
              horizon =0;
            }
        }
       else if (code=='\n')
        {
          value = converting_char_array_into_float();
          put_value_in_array(value,select_array);
          select_array=0;
          count++;
          horizon = 0;
        }
       else
        {
          put_value_in_buffer(code);
        }
     }while(code != EOF);
  sma();
  for(int i=0;i<count;i++)
    {
      for(int j=0;j<19;j++)
         {
           printf("%c",Date[i][j]);
         }
          printf("\t%f\n",smaa[i]);
    }
  return 0;
}

In [ ]:
"""using namespace std;

typedef struct {
    double cond;
		int a[5];
} matl1;

__device__  void foo(matl1 *d_matl1) {
    d_matl1->cond = 420.5;
    
	d_matl1->a[0]=5;
	d_matl1->a[1]=6;
	d_matl1->a[2]=7;
	d_matl1->a[3]=8;
	d_matl1->a[4]=9;
}

__global__ void myKernel(matl1 *d_matl1){
	foo(d_matl1);
	printf("Device is %.2f\n",d_matl1->cond);
}
__global__ void myKernel2(matl1 *d_matl1){
	//foo(d_matl1);
	printf("Device2 is %.2f\n",d_matl1->cond);
	printf("  a[0] = %d\n",d_matl1->a[0]);
}

int main() {

	matl1 *h_matl1, *d_matl1;

	h_matl1 = (matl1*)malloc(sizeof(matl1));
    cudaMalloc((void**)&d_matl1, sizeof(matl1));
	
	myKernel  <<<1,1 >>>(d_matl1);
	myKernel2  <<<1,1 >>>(d_matl1);
    cudaThreadSynchronize();

	cudaMemcpy(h_matl1, d_matl1, sizeof(matl1), cudaMemcpyDeviceToHost);

	printf("  Host is %.2f\n",h_matl1->cond);
	printf("  a[0] = %d\n",h_matl1->a[0]);
	cin.ignore();
    return 0;
}"""
#SUM REDUCTION
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <iostream>
#include <numeric>
using namespace std;

__global__ void sum(int* input)
{
	const int tid = threadIdx.x;

	auto step_size = 1;
	int number_of_threads = blockDim.x;

	while (number_of_threads > 0)
	{
		if (tid < number_of_threads) // still alive?
		{
			const auto fst = tid * step_size * 2;
			const auto snd = fst + step_size;
			input[fst] += input[snd];
		}

		step_size <<= 1; 
		number_of_threads >>= 1;
	}
 printf("SUMATION =  %d",input[0]);
}

int main()
{
	const auto count = 8;
	const int size = count * sizeof(int);
	int h[] = {13, 27, 15, 14, 33, 2, 24, 6};

	int* d;
	
	cudaMalloc(&d, size);
	cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);

	sum <<<1, count / 2 >>>(d);

	int result;
	cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);

	cout << "Sum is " << result << endl;

	getchar();

	cudaFree(d);
	delete[] h;

	return 0;
}

SyntaxError: ignored

In [ ]:
%%cu
#include<stdio.h>
char buffer[20];
char Date[10000][25];
float open[10000];
float low[10000];
float high[10000];
float close[10000];
float adj_close[10000];
float volm[10000];
int count = 0,horizon=0; 
int num_of_periods_for_mov_avg = 10;
float sma[10000]={0};	
float ema[10000]={0};
float wma[10000]={0};
float mfr[1000]={0.0},mfi[1000]={0.0},rsi[1000]={0.0},rs[1000] = {0.0};
float tr[1000] = {0.0},atr[1000] = {0.0};
int weight_array[40] = {0};
float expa=2;
__global__ void add(int a, int b, int *c)
{
 *c = a + b;
}

__global__ void smaaCuda()
{
    printf("smaaCuda");
}
void smaa()
 {
   int j;
   float avg = 0;
   for(int i=0;i<count-num_of_periods_for_mov_avg+1;i++)
     {
       avg = 0;
       for(j=i;j<i+num_of_periods_for_mov_avg;j++)
          {
            avg = avg+close[j];
          }
        sma[j-1] = avg/num_of_periods_for_mov_avg;
     }
 }

void emaa()
  {
    expa = expa/(num_of_periods_for_mov_avg);
    ema[9] = sma[9];
    for(int i=10;i<count;i++)
     {
       ema[i] = close[i]*expa + ema[i-1] * (1-expa); 
     }
    printf("\n\n");
  }



int weight_array_initialise()
  {
   int sum;
   for(int i=0;i<num_of_periods_for_mov_avg;i++)
     {
       weight_array[i] = i;
     }
   for(int i=0;i<num_of_periods_for_mov_avg;i++)
     sum = sum + weight_array[i];
   return sum;
  }

void wmaa()
 {
   int j,k,weight_array_sum;
   weight_array_sum = weight_array_initialise();
   float avg = 0;
   for(int i=0;i<count-num_of_periods_for_mov_avg+1;i++)
     {
       avg = 0;
       for(j=i,k=0;j<i+num_of_periods_for_mov_avg;j++,k++)
          {
            avg = avg+close[j]*weight_array[k];
          }
        wma[j-1] = avg/weight_array_sum;
     }
 }

void print()
  {
    printf("SMA\t\tEMA\t\tWMA\t\tRelative Strngth   RSI \t\tTR\t\tATR\t\tMFR\t\tMFI\n\n");
    for(int i=0;i<count;i++)
     {
      printf("%f\t",sma[i]);
      printf("%f\t",ema[i]);
      printf("%f\t",wma[i]);
      printf("%f\t%f\t",rs[i],rsi[i]);
      printf("%f\t%f\t",tr[i],atr[i]);
      printf("%f\t%f\n",mfr[i],mfi[i]);
     }
  }

float finding_max(float hl,float hc,float lc)
 {
   if(hl>=hc && hl>=lc)
    return hl;
  else if(hc>=hl && hc>=lc)
    return hc;
  else
    return lc;
 }

void setting_value_for_dm(float *dm1,float *dm2)
  {
    if(*dm1<0 && *dm2<0)
      {
        *dm1 = 0;
        *dm2 = 0;
      }
     else if(*dm1>*dm2)
       *dm2 = 0;
     else
       *dm1 = 0;
  }

void MFI()
 {
   int i,j;
   float typical_price[1000]={0.0},mon_flow[1000]={0.0};
   float pos_mon_flow=0.0,neg_mon_flow=0.0;
   for(i=0;i<count;i++)
    {
      typical_price[i] = (high[i]+low[i]+close[i])/3;
      mon_flow[i] = typical_price[i] * (volm[i] /10000);
    }
   for(j=0;j<count-14;j++)
    {
      pos_mon_flow=0.0;
      neg_mon_flow=0.0;
     for(i=j+1;i<j+15;i++)
      {
       {
         if(typical_price[i-1]>typical_price[i])
           neg_mon_flow += mon_flow[i];
         else
           pos_mon_flow += mon_flow[i];
        }
      }
       mfr[i-1] = pos_mon_flow/neg_mon_flow;
       mfi[i-1] = (100*mfr[i-1]) / (1+mfr[i-1]);
    }
 }

void RSI()
 {
   float change[1000] = {0.0},up_mov[1000] = {0.0}, down_mov[1000] = {0.0},up_movi=0.0,down_movi=0.0;
   int i,j;
   for(i=1;i<count;i++)
    {
      change[i] = close[i] - close [i-1];
      if(change[i]>0)
        up_mov[i] = change[i];
      else
	down_mov[i] = -1.0 * change[i];
    }
   for(i=1;i<15;i++)
      {
         up_movi += up_mov[i];
         down_movi += down_mov[i];
      }
   up_movi = up_movi/14;
   down_movi = down_movi/14;
   rs[i-1] = up_movi/down_movi;
   float tp1 = 1+rs[i-1];
   float tp2 = 100/tp1;
   rsi[i-1] = 100 - tp2;
   //printf("\n\n");
   for(j=15;j<count;j++)
    {
     up_movi = (up_movi * 13 + up_mov[j])/14.0;
     down_movi = (down_movi * 13 + down_mov[j])/14.0;
     rs[j] = up_movi/down_movi;
     tp1 = 1+rs[j];
     tp2 = 100.0/tp1;
     rsi[j] = 100.0 - tp2;
    }
   //printf("\n\n");
   //for(i=0;i<count;i++)
     
 }

void ATR()
  {
  int i,j;
  float hl,hc,lc,sum = 0.0;
  for(i=1;i<count;i++)
  {
    hl = high[i]-low[i];
    hc = high[i]-close[i-1];
    lc = low[i]-close[i-1];
    tr[i] = finding_max(hl,hc,lc);
  }
  for(i=0;i<count-14;i++)
   {
     for(j=i+1;j<i+15;j++)
       {
        sum = sum+tr[j];
       }
     atr[j-1] = sum/14;
     sum = 0.0;
   }
  //for(i=0;i<count;i++)
    
  printf("\n\n");   
  }


void ADX()
 {
  float plus_dm,minus_dm,hc,hl,lc,tr;
  plus_dm = high[2] - high[1];
  minus_dm = low[1] - low[2];
  //setting_value_for_dm();
  hl = high[2]-low[2];
  hc = high[2]-close[1];
  lc = low[2]-close[1];
  tr = finding_max(hl,hc,lc);
  setting_value_for_dm(&plus_dm,&minus_dm);
 }

void put_value_in_array(float value,int select_array)
   {
     switch(select_array)
       {
         case 1:
           open[count] = value;
           break;
         case 2:
           high[count] = value;
           break;
         case 3:
           low[count] = value;
           break;
         case 4:
           close[count] = value;
           break;
         case 5:
           volm[count] = value;
           break;
       }
   }


//while for confusion
float converting_char_array_into_float()
 {
   float num=0,num1,num2;
   int i=0,mul=1;
   for(int j=0;j<horizon;j++)
   {
       if(buffer[i]=='.')
         break;
       //printf("%c",buffer[j]);
      num2 = (float)buffer[i]-48;
      num = num*10+num2;
      i++;
      if(buffer[i]==13)
      {
       return num;
      }
   }
   i++;
   while(buffer[i]!='\0')
    {
     num2=(float)buffer[i]-48;
     mul=mul*10;
     num1=num2/mul;
     num=num+num1;
     i++;
    }
  
   return num;
 }
 /*
float converting_char_array_into_float()
 {
   float num=0,num1,num2;
   int i=0,mul=1;
   while(buffer[i]!='.')
    {
     num2 = (float)buffer[i]-48;
     num = num*10+num2;
     i++;
     if(buffer[i]==13)
      {
       return num;
      }
    }
   i++;
   while(buffer[i]!='\0')
    {
     num2=(float)buffer[i]-48;
     mul=mul*10;
     num1=num2/mul;
     num=num+num1;
     i++;
    }
   return num;
 }
*/
void reinitialise_buffer_to_zero()
  {
    for(int i=0;i<20;i++)
      {
	buffer[i] = '\0';
      }
  }

void put_value_in_buffer(char code)
  {
    buffer[horizon] = code;
    horizon++;
  }

void put_in_date_array()
   {
     for(int i=0;i<20;i++)
      {
	Date[count][i] = buffer[i]; 
      }
   }

int main()
  {
    printf("Hello");
    FILE *file;
    float value;
    int a,b,c;
    int *dev_c;
    a=3;
    b=4;
    int select_array=0;
    char code;
    file = fopen("gdrive/My Drive/export_csv1.csv","r");
    code = (char)fgetc(file);
    while(code!='\n')
       code = (char)fgetc(file);
    do
     {
       code = (char)fgetc(file);
       if(code==',')
        {
          if(select_array==0)
             {
		select_array++;
                put_in_date_array();
                reinitialise_buffer_to_zero();
                horizon = 0;
             }
          else
            {
              value = converting_char_array_into_float();
              put_value_in_array(value,select_array);
              select_array++;
              reinitialise_buffer_to_zero();
              horizon =0;
	    }
        }
       else if (code=='\n')
        {
          value = converting_char_array_into_float();
          put_value_in_array(value,select_array);
          select_array=0;
          count++;
          horizon = 0;
        }
       else
        {
       	  put_value_in_buffer(code);
        }
     }while(code != EOF);
    for(int i=0;i<count;i++)
     {
       for(int j=0;j<25;j++)
         {
           printf("%c",Date[i][j]);
         }
       printf("\t");
       printf("%f\t",open[i]);
       printf("%f\t",high[i]);
       printf("%f\t",low[i]);
       printf("%f\t",close[i]);
       printf("%f\n",volm[i]);
    }
   printf("\n\n");
   smaa();
   emaa();
   wmaa();
   //ADX();
   MFI();
   RSI();
   ATR();
   print();
   //smaaCuda<<<1,1>>>();
   printf("Hello");
   smaaCuda<<<1,1>>>();
   cudaMalloc((void**)&dev_c, sizeof(int));
   add<<<1,1>>>(a,b,dev_c);
   cudaMemcpy(&c, dev_c, sizeof(int), cudaMemcpyDeviceToHost);
   printf("%d + %d is %d\n", a, b, c);
   cudaFree(dev_c);
   return 0;
  }

Hello10/14/2019 9:15          	1356.850098	1360.850098	1353.099976	1354.300049	412830.000000
10/14/2019 9:25          	1354.400024	1355.699951	1352.599976	1354.500000	200903.000000
10/14/2019 9:35          	1354.250000	1355.500000	1353.050049	1354.699951	140881.000000
10/14/2019 9:45          	1354.699951	1357.000000	1353.699951	1356.750000	112477.000000
10/14/2019 9:55          	1356.750000	1358.000000	1354.000000	1355.850098	130328.000000
10/14/2019 10:05         	1355.850098	1359.000000	1355.500000	1357.150024	104836.000000
10/14/2019 10:15         	1357.150024	1358.300049	1356.199951	1357.800049	69588.000000
10/14/2019 10:25         	1357.900024	1358.300049	1357.000000	1357.850098	78146.000000
10/14/2019 10:35         	1357.850098	1358.350098	1357.000000	1357.550049	59662.000000
10/14/2019 10:45         	1357.550049	1358.150024	1356.050049	1357.000000	74751.000000
10/14/2019 10:55         	1357.000000	1362.500000	1357.000000	1362.500000	164301.000000
10/14/2019 11:05         	1362.

In [ ]:
%%cu
#include <pthread.h>
#include <stdio.h>

const int N = 1 << 20;

__global__ void kernel(float *x, int n)
{
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    printf("Tid = %d",tid);
   /* for (int i = tid; i < n; i += blockDim.x * gridDim.x) {
        x[i] = sqrt(pow(3.14159,i));
    }*/
}

void *launch_kernel(void *dummy)
{
    float *data;
    static int cnt=0;
    cudaMalloc(&data, N * sizeof(float));
    printf("\nLaunch Kernel %d",cnt);
    cnt++;
    kernel<<<1, 64>>>(data, N);

    cudaStreamSynchronize(0);

    return NULL;
}

int main()
{
    const int num_threads = 8;

    pthread_t threads[num_threads];

    for (int i = 0; i < num_threads; i++) {
        if (pthread_create(&threads[i], NULL, launch_kernel, 0)) {
            fprintf(stderr, "Error creating threadn");
            return 1;
        }
    }

    for (int i = 0; i < num_threads; i++) {
        if(pthread_join(threads[i], NULL)) {
            fprintf(stderr, "Error joining threadn");
            return 2;
        }
    }

    cudaDeviceReset();

    return 0;
}


Launch Kernel 0
Launch Kernel 1
Launch Kernel 2Tid = 0Tid = 1Tid = 2Tid = 3Tid = 4Tid = 5Tid = 6Tid = 7Tid = 8Tid = 9Tid = 10Tid = 11Tid = 12Tid = 13Tid = 14Tid = 15Tid = 16Tid = 17Tid = 18Tid = 19Tid = 20Tid = 21Tid = 22Tid = 23Tid = 24Tid = 25Tid = 26Tid = 27Tid = 28Tid = 29Tid = 30Tid = 31Tid = 32Tid = 33Tid = 34Tid = 35Tid = 36Tid = 37Tid = 38Tid = 39Tid = 40Tid = 41Tid = 42Tid = 43Tid = 44Tid = 45Tid = 46Tid = 47Tid = 48Tid = 49Tid = 50Tid = 51Tid = 52Tid = 53Tid = 54Tid = 55Tid = 56Tid = 57Tid = 58Tid = 59Tid = 60Tid = 61Tid = 62Tid = 63Tid = 32Tid = 33Tid = 34Tid = 35Tid = 36Tid = 37Tid = 38Tid = 39Tid = 40Tid = 41Tid = 42Tid = 43Tid = 44Tid = 45Tid = 46Tid = 47Tid = 48Tid = 49Tid = 50Tid = 51Tid = 52Tid = 53Tid = 54Tid = 55Tid = 56Tid = 57Tid = 58Tid = 59Tid = 60Tid = 61Tid = 62Tid = 63Tid = 0Tid = 1Tid = 2Tid = 3Tid = 4Tid = 5Tid = 6Tid = 7Tid = 8Tid = 9Tid = 10Tid = 11Tid = 12Tid = 13Tid = 14Tid = 15Tid = 16Tid = 17Tid = 18Tid = 19Tid = 20Tid = 21Tid = 22Tid = 23Tid = 24Tid 